<a href="https://colab.research.google.com/github/sunwoosf/Securing-Applications-in-Node.js-V-/blob/master/Lm2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install deepspeed
!pip install accelerate
!pip install sentencepiece
!pip install langchain
!pip install torch
!pip install einops
!pip install bitsandbytes
!pip install xformers
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 21.3 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.9.5-py3-none-any.whl size=844534 sha256=dc927eb262f26a6385fa407369100592644fe3669d28b9dd797ffb6046ff4802
  Stored in directory: /root/.cache/pip/wheels/7e/a9/bb/a00d383521da14dc91b65ae2d0062401b750d968a548401b2a
Successfully built deepspeed
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
import deepspeed
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

[2023-06-28 13:31:15,179] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
model_name  = 'chavinlo/alpaca-native'
model       = LlamaForCausalLM.from_pretrained(
                model_name,
                load_in_8bit    = True,
                device_map      =  'auto'
                )
tokenizer   = LlamaTokenizer.from_pretrained(model_name, device_map='auto')


In [ ]:
pipe = pipeline(
    'text-generation',
    model               = model,
    tokenizer           = tokenizer,
    max_length          = 500,
    temperature         = 0.3,
    top_p               = 0.95,
    repetition_penalty  = 1.2
)

template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} from {Question}.
"""
prompt      = PromptTemplate(template=template, input_variables=['Table', 'Question', 'Columns'])
local_llm   = HuggingFacePipeline(pipeline=pipe)
llm_chain   = LLMChain(prompt=prompt, llm=local_llm)


def get_llm_response(llm_chain, table, columns, question):
    return llm_chain.run({ 'Table': table, 'Columns': columns, 'Question': question })

# Example 1

In [ ]:
table       = 'employee'
columns     = ['id', 'name', 'date_of_birth', 'band', 'manager_id']
#
question    = 'Query the count of employees in band L6 with 239045 s the manager ID'
answer      = get_llm_response(llm_chain, table, columns, question)
#
print(f'Question: {question}')
print(f'Answer  : {answer}')

# Example 2

In [ ]:
table       = 'yfinance.equity_ohlcv'
columns     = ['date', 'symbol', 'open', 'high', 'low', 'close', 'dividend', 'split']
#
question    = ' give me daily open, HIGH, low,  and close prices for apple stock?'
answer      = get_llm_response(llm_chain, table, columns, question)
#
print(f'Question: {question}')
print(f'Answer  : {answer}')

# Example 3

In [ ]:
table       = 'yfinance.equity_ohlcv'
columns     = ['date', 'symbol', 'open', 'high', 'low', 'close', 'dividend', 'split']
#
question    = ' give me daily ohlc for apple stock?'
answer      = get_llm_response(llm_chain, table, columns, question)
#
print(f'Question: {question}')
print(f'Answer  : {answer}')